<a href="https://colab.research.google.com/github/gomes-lab/course-cs2770-2023sp/blob/main/AMPL_SaveIt_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies
!pip install -q amplpy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 29.7 MB/s eta 0:00:00


In [2]:
# Import packages and setup AMPL
from amplpy import AMPL, tools
ampl = tools.ampl_notebook(
    modules=["highs"],      # modules to install
    license_uuid="default", # license to use
    g=globals())            # instantiate AMPL object and register magics

Using default Community Edition License for Colab. Get yours at: https://ampl.com/ce
Licensed to AMPL Community Edition License for the AMPL Model Colaboratory (https://colab.ampl.com).


In [3]:
# Model for the SaveIt example shown in class
%%ampl_eval
set Grade;
set Material;

var X {Grade, Material} >= 0;

maximize Profit: 5.5 * (X['A',1] + X['A', 2] + X['A', 3] + X['A', 4]) + 4.5 * (X['B',1] + X['B', 2] + X['B', 3] + X['B', 4]) + 3.5 * (X['C',1] + X['C', 2] + X['C', 3] + X['C', 4]);
subject to A1_const: X['A', 1] <= 0.3 * sum {j in Material} X['A',j];
subject to A2_const: X['A', 2] >= 0.4 * sum {j in Material} X['A',j];
subject to A3_const: X['A', 3] <= 0.5 * sum {j in Material} X['A',j];
subject to A4_const: X['A', 4] = 0.2 * sum {j in Material} X['A',j];
subject to B1_const: X['B', 1] <= 0.5 * sum {j in Material} X['B',j];
subject to B2_const: X['B', 2] >= 0.1 * sum {j in Material} X['B',j];
subject to B4_const: X['B', 4] = 0.1 * sum {j in Material} X['B',j];
subject to C1_const: X['C', 1] <= 0.7 * sum {j in Material} X['C',j];
subject to Mat1_Max: X['A', 1] + X['B', 1] + X['C', 1] <= 3000;
subject to Mat2_Max: X['A', 2] + X['B', 2] + X['C', 2] <= 2000;
subject to Mat3_Max: X['A', 3] + X['B', 3] + X['C', 3] <= 4000;
subject to Mat4_Max: X['A', 4] + X['B', 4] + X['C', 4] <= 1000;
subject to Mat1_Min: X['A', 1] + X['B', 1] + X['C', 1] >= 1500;
subject to Mat2_Min: X['A', 2] + X['B', 2] + X['C', 2] >= 1000;
subject to Mat3_Min: X['A', 3] + X['B', 3] + X['C', 3] >= 2000;
subject to Mat4_Min: X['A', 4] + X['B', 4] + X['C', 4] >= 500;
subject to Treat_Cost: 3 * (X['A', 1] + X['B', 1] + X['C', 1]) + 6 * (X['A', 2] + X['B', 2] + X['C', 2]) + 4 * (X['A', 3] + X['B', 3] + X['C', 3]) + 5 * (X['A', 4] + X['B', 4] + X['C', 4]) <= 30000;


In [4]:
# Define the sets and paramter values
ampl.set['Grade'] = ['A', 'B', 'C']
ampl.set['Material'] = [1, 2, 3, 4]

In [5]:
# Alternate formulation of the Save-It Problem using parameters and sums
# %%ampl_eval
# set Grade;
# set Material;

# param profit_per_lb {Grade} >= 0;
# param treatment_cost {Material} >= 0;
# param lbs_avail {Material} >= 0;

# var X {Grade, Material} >= 0;

# maximize Profit: sum {i in Grade} profit_per_lb[i] * sum {j in Material} X[i,j];
# subject to Mat_Avail {j in Material}: sum {i in Grade} X[i,j] <= lbs_avail[j];
# subject to Min_Treat {j in Material}: sum {i in Grade} X[i,j] >= 0.5 * lbs_avail[j]; 
# subject to Treat_Cost: sum {j in Material} treatment_cost[j] * sum {i in Grade} X[i,j] = 30000; 
# subject to A1_const: X['A', 1] <= 0.3 * sum {j in Material} X['A',j];
# subject to A2_const: X['A', 2] >= 0.4 * sum {j in Material} X['A',j];
# subject to A3_const: X['A', 3] <= 0.5 * sum {j in Material} X['A',j];
# subject to A4_const: X['A', 4] = 0.2 * sum {j in Material} X['A',j];
# subject to B1_const: X['B', 1] <= 0.5 * sum {j in Material} X['B',j];
# subject to B2_const: X['B', 2] >= 0.1 * sum {j in Material} X['B',j];
# subject to B4_const: X['B', 4] = 0.1 * sum {j in Material} X['B',j];
# subject to C1_const: X['C', 1] <= 0.7 * sum {j in Material} X['C',j];

In [6]:
# Define the sets and parameter values for the more compact model
# ampl.set['Grade'] = ['A', 'B', 'C']
# ampl.set['Material'] = [1, 2, 3, 4]

# ampl.param['profit_per_lb'] = [5.5, 4.5, 3.5]
# ampl.param['treatment_cost'] = [3, 6, 4, 5]
# ampl.param['lbs_avail'] = [3000, 2000, 4000, 1000]

In [7]:
# Solve
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=0"
ampl.solve()

HiGHS 1.5.1: tech:outlev=0
HiGHS 1.5.1: optimal solution; objective 35109.64912
6 simplex iterations
0 barrier iterations


In [8]:
# Check variables
ampl.var["X"].get_values().to_pandas()

,X.val
"(A, 1.0)",412.280702
"(A, 2.0)",859.649123
"(A, 3.0)",447.368421
"(A, 4.0)",429.824561
"(B, 1.0)",2587.719298
"(B, 2.0)",517.543860
"(B, 3.0)",1552.631579
"(B, 4.0)",517.543860
"(C, 1.0)",0.000000
"(C, 2.0)",0.000000
